In [1]:
import xarray as xr
from xhistogram.xarray import histogram
import richdem as rd
import rioxarray as rxr
import xrspatial as xrs
import numpy as np
import pandas as pd
import geopandas as gpd
import glob
from scipy import stats
import matplotlib.pyplot as plt

%matplotlib inline

import pysolar
from datetime import datetime

from aster_utils import mapZonalStats

from asp_binder_utils import get_dem

import warnings
warnings.filterwarnings('ignore')

/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/spestana/opt/anaconda3/envs/goes-linux/lib/python3.6/site-packages/osgeo/gdal.py:107: DeprecationWarning: gdal.py was placed in a namespace, it is now available as osgeo.gdal
  DeprecationWarning)


In [2]:
# Larger area around Tuolumne further south and east to include Mammoth
(utm_e_UL, utm_n_UL) = 282500, 4205000
(utm_e_UR, utm_n_UR) = 305000, 4205000
(utm_e_LR, utm_n_LR) = 327500, 4160000
(utm_e_LL, utm_n_LL) = 305000, 4160000

geometry = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [utm_e_UL, utm_n_UL],
            [utm_e_UR, utm_n_UR],
            [utm_e_LR, utm_n_LR],
            [utm_e_LL, utm_n_LL]
        ]]
    }
]

---

In [3]:
ds = xr.open_dataset('/storage/spestana/ASTER-MODIS-GOES-DEM-FVEG_v4_orthorectified.nc')

---

In [4]:
modscag_filepaths = glob.glob('/storage/MODIS/Tuolumne_MOD09GA/*.tif')

In [5]:
modscag_da_list = []
for modscag_filepath in modscag_filepaths:
    _da = xr.open_rasterio(modscag_filepath)
    _da_masked = _da.where(_da <= 100)
    _da_masked = _da_masked.squeeze().drop('band')
    this_time = pd.Timestamp(datetime.strptime(_da_masked.attrs['TIFFTAG_DATETIME'],'%Y:%m:%d %H:%M:%S'))#np.datetime64(datetime.strptime(_da_masked.attrs['TIFFTAG_DATETIME'],'%Y:%m:%d %H:%M:%S'))
    _da_masked = _da_masked.assign_coords({'time':this_time})
    modscag_da_list.append(_da_masked.rio.reproject_match(ds))
    
# manually picking the ones I need to match my GOES/ASTER/MODIS dataset (including duplicates where we had 2 ASTER in one day)
idx = [2,3,4,4,5,10,11,12,20,21,21,22,23,24,26,26,29,29,34,34,36,36,37,38,39,40,41]
new_modscag_da_list = [modscag_da_list[i] for i in idx]

In [6]:
modscag_da = xr.concat(new_modscag_da_list, dim='time')
modscag_da['time'] = ds.time.values # overwrite time values with ASTER times
ds['fsca'] = modscag_da # add to main dataset

---

In [8]:
ds.to_netcdf('/storage/spestana/ASTER-MODIS-GOES-DEM-FVEG-FSCA_v5_orthorectified.nc')